<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 1182, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (345/345), done.
remote: Total 1182 (delta 30), reused 322 (delta 5), pack-reused 832
Receiving objects: 100% (1182/1182), 195.63 MiB | 25.36 MiB/s, done.
Resolving deltas: 100% (107/107), done.
Checking out files: 100% (942/942), done.


In [2]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'ผักกระเฉด','ผักกุยช่าย','ผักขึ้นช่าย','ผักชะอม','ผักชี','ผักชีฝรั่ง','ผักตำลึง',
               'มะปราง','มะพลับ','มะละกอ','มะกรูด','มะเขือพวง','มะเขือเทศ','มะระ','มะรุม','มะตูม','มันแกว1',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด','พุทรา','เพกา',
               'ฝรั่ง',
               'ฟัก','ฟักทอง',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [3]:
import pandas as pd
from glob import glob
import librosa

for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [4]:
df = pd.DataFrame.from_dict(source)
df

,audio_time,sampling_rate,feature_extraction,label,description
0,"[0.00015258789, 0.00022888184, 0.00022888184, ...",22050,"[[-638.9144, -641.23175, -641.37787, -636.0402...",0,กระชาย
1,"[-0.0009765625, -0.001449585, -0.001373291, -0...",22050,"[[-646.73065, -649.3781, -651.3155, -651.3928,...",0,กระชาย
2,"[-0.0001373291, -0.00018310547, -4.5776367e-05...",22050,"[[-655.651, -653.5817, -654.66907, -655.0482, ...",0,กระชาย
3,"[-0.00024414062, -0.0002593994, -0.00022888184...",22050,"[[-650.5033, -649.7863, -647.1397, -648.9082, ...",0,กระชาย
4,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
...,...,...,...,...,...
935,"[-0.00016784668, -0.00015258789, -0.0001983642...",22050,"[[-668.4974, -670.4946, -672.10724, -671.17737...",47,สับปะรด
936,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-666.57153, -663.4788, -662.98236, -663.7374...",47,สับปะรด
937,"[0.0, 0.0, 0.0, 3.0517578e-05, -4.5776367e-05,...",22050,"[[-644.81506, -644.89075, -644.7345, -643.6741...",47,สับปะรด
938,"[-0.00018310547, -0.000289917, -0.00022888184,...",22050,"[[-614.8528, -614.2113, -613.9889, -613.8524, ...",47,สับปะรด


In [5]:
import numpy as np
import pandas as pd

Test = pd.DataFrame(columns=source.keys())

# create Test model
for description in np.unique(df['description']):
    imp = df[df['description']==description].sample(5,random_state=1)
    Test = pd.concat([Test,imp], axis=0)

# create Train model
Train = df.drop(Test.index)

X_train, X_test = np.array(Train['feature_extraction'].to_list()), np.array(Test['feature_extraction'].to_list())
y_train, y_test = np.array(Train['label'].to_list()), np.array(Test['label'].to_list())

In [6]:
import tensorflow as tf
from tensorflow import keras 
np.random.seed(42)
tf.random.set_seed(42)

model = keras.Sequential()
model.add(keras.layers.Conv2D(128, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(128, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
22/22 [==============================] - 32s 23ms/step - loss: 31.2115 - accuracy: 0.0157
Epoch 2/100
22/22 [==============================] - 0s 14ms/step - loss: 3.6856 - accuracy: 0.0671
Epoch 3/100
22/22 [==============================] - 0s 14ms/step - loss: 3.0919 - accuracy: 0.1914
Epoch 4/100
22/22 [==============================] - 0s 14ms/step - loss: 2.4903 - accuracy: 0.3214
Epoch 5/100
22/22 [==============================] - 0s 14ms/step - loss: 2.0380 - accuracy: 0.4457
Epoch 6/100
22/22 [==============================] - 0s 14ms/step - loss: 1.6087 - accuracy: 0.5243
Epoch 7/100
22/22 [==============================] - 0s 14ms/step - loss: 1.4348 - accuracy: 0.5886
Epoch 8/100
22/22 [==============================] - 0s 14ms/step - loss: 1.3085 - accuracy: 0.6071
Epoch 9/100
22/22 [==============================] - 0s 14ms/step - loss: 0.9355 - accuracy: 0.6986
Epoch 10/100
22/22 [==============================] - 0s 14ms/step - loss: 0.9377 - accuracy: 0.70

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 255, 128)       3328      
_________________________________________________________________
dropout (Dropout)            (None, 9, 255, 128)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 127, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 127, 128)       409728    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 127, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 63, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16128)             0

In [8]:
model.evaluate(X_test[:,:,:,None],y_test)

8/8 [==============================] - 0s 23ms/step - loss: 0.5853 - accuracy: 0.9333


[0.585340142250061, 0.9333333373069763]

In [9]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(np.round(model.predict(X_test[:,:,:,None])),axis=1)
target_names = [f"Class {label}" for label in range(len(np.unique(source['label'])))]

print(classification_report(y_test, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.56      1.00      0.71         5
     Class 1       0.80      0.80      0.80         5
     Class 2       1.00      1.00      1.00         5
     Class 3       1.00      0.80      0.89         5
     Class 4       1.00      0.80      0.89         5
     Class 5       1.00      1.00      1.00         5
     Class 6       1.00      1.00      1.00         5
     Class 7       0.83      1.00      0.91         5
     Class 8       1.00      1.00      1.00         5
     Class 9       1.00      1.00      1.00         5
    Class 10       0.83      1.00      0.91         5
    Class 11       1.00      0.80      0.89         5
    Class 12       1.00      0.80      0.89         5
    Class 13       1.00      0.80      0.89         5
    Class 14       1.00      1.00      1.00         5
    Class 15       1.00      0.40      0.57         5
    Class 16       1.00      1.00      1.00         5
    Class 17       0.83    